In [90]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
from pymystem3 import Mystem
m = Mystem()

> Здесь можно прочитать краткий ход действий, а ниже посмотреть код с комментариями. 



## Наш бейзлайн:
Precision - 0.13 

Recall - 0.24 

F1 - 0.16 

Jaccard - 0.09

# Результат работы:

Удалось повысить F-score до 0.2

# #1 Способ (failed)
- выбрать последний вариант разбора pymorphy вместо первого + 0,01 (как выяснилось, это исключает числительные)

Вот так упал результат:

Precision -  0.12

Recall -  0.23

F1 -  0.15

Jaccard -  0.09


# #2 Cпособ 2 (success!)
- возьмем меньше ключевых слов (6) и получим уже F-score 0.17:

```sh
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6: -1]] 

```

Precision -  0.19

Recall -  0.18

F1 -  0.17

Jaccard -  0.11

# #3 способ (success!)
- возьмем униграммы, F-score повысился еще немного:

Precision -  0.19

Recall -  0.18

F1 -  0.18

Jaccard -  0.11

# #4 способ (success!)
- уменьшим минимальное кол-во документов: min_df=5 -> min_df=2

Precision -  0.2

Recall -  0.19

F1 -  0.19

Jaccard -  0.11

# #5 Способ (success!)
- объединить заголовок и текст и уменьшим min_df до 1 (ну да, здесь лучше сработало 1) :

```sh
data['whole_text'] = data['content_norm'] + data['title_norm']
```

Precision -  0.21

Recall -  0.2

F1 -  0.2

Jaccard -  0.12



# #6 Способ (fail)
- я улучшила результат gensim графа тем, что разбила н-граммы на биграммы (простым циклом, потому что так и не поняла, как просить у гензима униграммы)
В результате F-score значительно вырос (с 0.09 до 0.15), но все равно не перешагнул наш бейзлайн. Код можно посмотреть внизу.



In [2]:
pd.set_option('display.max_colwidth', 1000)

In [9]:
PATH_TO_DATA = '/Users/Marina/PycharmProjects/compling_nlp_hse_course/ru_kw_eval_datasets/data/ng'

In [13]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [20]:
files[2]

'/Users/Marina/PycharmProjects/compling_nlp_hse_course/ru_kw_eval_datasets/data/ng/ng_1.jsonlines.zip'

In [ ]:
data = files[2]

In [23]:
data = pd.read_json(files[2], lines=True)

In [24]:
data.shape

(988, 5)

In [26]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

- ## Здесь я неудачно пыталась брать последний разбор пайморфи:

In [159]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']
    return words

#morph.parse('привет я марина у меня есть сто рублей')
analysis = morph.parse('сто')
for i, word in enumerate(analysis):
    if word.tag.POS == 'NUMR': #.normal_form()
        print (word.tag.POS)
        print (analysis[i])

In [139]:
#m.analyze('иванов')

In [140]:
#'Surn' in morph.parse('сто')[0][1]
#morph.parse('двести')[-1]

In [160]:
data['content_norm'] = data['content'].apply(normalize)

In [161]:
data['title_norm'] = data['title'].apply(normalize)

- ## Вот здесь я объединила текст и заголовок:

In [393]:
data['whole_text'] = data['content_norm'] + data['title_norm']

In [150]:
#kw1

- ## Здесь поменяла биграммы на униграммы и уменьшина min_df (мин кол-во документов, в кот встречается слово) сначала до 2, потом до 1:

In [686]:
tfidf = TfidfVectorizer(ngram_range=(1, 1), min_df=1)

In [687]:
#data['content_norm_str'] = data['content_norm'].apply(' '.join)
data['whole_text_norm_str'] = data['whole_text'].apply(' '.join)

In [688]:
#tfidf.fit(data['content_norm_str'])
tfidf.fit(data['whole_text_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [689]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [690]:
#texts_vectors = tfidf.transform(data['content_norm_str'])
texts_vectors = tfidf.transform(data['whole_text_str'])

- ## Здесь уменьшила кол-во ключевых слов с 10 до 6:

In [691]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6: -1]] 

In [692]:
evaluate(data['keywords'], keywords)

Precision -  0.21
Recall -  0.2
F1 -  0.2
Jaccard -  0.12


# Вот тут я разбила н-граммы на униграммы (способ 6):

In [704]:
from gensim.summarization import keywords

In [ ]:
Precision -  0.08
Recall -  0.11
F1 -  0.08
Jaccard -  0.05

In [ ]:
Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09

In [722]:
gensim_kws = data['whole_text'].apply(lambda x: keywords(' '.join(x)).split('\n')[:7])

In [723]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.08
Recall -  0.1
F1 -  0.09
Jaccard -  0.05


In [724]:
gensim_kws2 = []
for i in gensim_kws:
    i = ' '.join([w for w in i])
    gensim_kws2.append(i.split(' '))

In [725]:
evaluate(data['keywords'], gensim_kws2)

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09
